In [1]:
# Module Imports
import pandas as pd

[     1      2      3 ... 138491 138492 138493]


In [9]:
# Constants
movie_data_csv = "C:/Users/kretz/OneDrive/DataSets/movie.csv"
rating_data_csv = "C:/Users/kretz/OneDrive/DataSets/rating.csv"
TITLE = 'title'

In [3]:
# Read Data
movie_df = pd.read_csv(movie_data_csv)
rating_df = pd.read_csv(rating_data_csv)

In [6]:
# Data Transformations etc.
users_id = rating_df["userId"].unique()
movies_id = movie_df["movieId"].unique()

[     1      2      3 ... 138491 138492 138493]


In [8]:
print(movie_df)

       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
27273   131254        Kein Bund für's Leben (2007)   
27274   131256       Feuer, Eis & Dosenbier (2002)   
27275   131258                  The Pirates (2014)   
27276   131260                 Rentun Ruusu (2001)   
27277   131262                    Innocence (2014)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                              

In [15]:
clean_movie_df = movie_df.copy()
clean_movie_df[TITLE] = clean_movie_df[TITLE].map(lambda x: x.strip()).map(lambda x: x.lower())